In [1]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import scipy.stats as stats
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
print(sys.version)

3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]


In [3]:
parent = os.path.dirname(os.getcwd())
ufo = pd.read_pickle(parent + '/data/cleaned_nuforc_reports_07202018.pkl') #UFO SIGHTINGS
airport = pd.read_csv(parent + '/data/airportData.csv') #AIRPORT LOCATIONS
military = pd.read_csv(parent + '/data/militaryData.csv') #MILITARY BASE LOCATIONS

In [4]:
ufo.head()

,Occurred,Reported,Posted,Location,Shape,Duration,Report,Sighting Time,Report Time,Latitude,Longitude,Shape Categories Revised
0,1996-06-24 00:30:00,1996-06-26 12:42:00,1999-11-02,"Aurora, CO",Changing,1 hour,Referred to Marilyn Childs to call CO state di...,00:30,12:42,39.729432,-104.831920,[Changing]
1,NaT,2006-03-21 12:11:00,2006-05-15,"Bristol, PA",Light,About 1 min.,"Hi, I am an avid sky watcher and usually I not...",NaN,12:11,40.100666,-74.851831,[Light]
2,2018-10-21 00:45:00,2015-05-16 00:34:00,2015-05-22,"Saskatoon (Canada), SK",Light,2 minutes,We seemed to know to look up and and for me it...,00:45,12:34,52.157331,-106.529930,"[Light, Circle]"
3,2018-10-21 01:00:00,2001-10-21 11:50:00,2001-11-20,Dublin,Light,a few seconds,"A ball of light, lighting up my pitch dark roo...",01:00,11:50,53.349764,-6.260273,[Light]
4,2018-10-21 01:00:00,1999-09-02 09:52:00,2003-02-05,"Careywood & Bayview, ID",Light,few minutes,"Ball of light similiar to reported ""Earht Ligh...",01:00,09:52,NaN,NaN,[Light]


In [18]:
#DROP UFO FEATURE COLUMNS WE WON'T BE USING IN THIS NOTEBOOK.
ufo = ufo.drop(['Report','Shape Categories Revised','Duration'], axis=1)

In [19]:
airport.head()

,Type,City,StateName,ARPLatitude,ARPLatitudeS,ARPLongitude,ARPLongitudeS,ActiviationDate
0,AIRPORT,ADAK ISLAND,ALASKA,51-53-00.8954N,186780.8954N,176-38-32.9277W,635912.9277W,04/01/1949
1,AIRPORT,AKHIOK,ALASKA,56-56-19.2870N,204979.2870N,154-10-57.2000W,555057.2000W,11/01/1979
2,AIRPORT,AKIACHAK,ALASKA,60-54-49.7150N,219289.7150N,161-29-35.9850W,581375.9850W,01/01/1968
3,SEAPLANE BASE,AKIACHAK,ALASKA,60-54-28.3130N,219268.3130N,161-26-06.2780W,581166.2780W,07/01/1966
4,AIRPORT,AKIAK,ALASKA,60-54-10.4300N,219250.4300N,161-13-50.1000W,580430.1000W,07/01/1966


In [20]:
def convert_tude(tude):
    multiplier = 1 if tude[-1] in ['N','E'] else -1
    return multiplier * np.sum(np.float(x)/(60**n) for n,x in enumerate(tude[:-1].split('-')))

In [21]:
#DROP NULL, RESET INDEX
airport = airport.dropna().reset_index(drop=True)

#CONVERT L-TUDE VALUES 
airport['Lon'] = airport['ARPLongitude'].apply(convert_tude)
airport['Lat'] = airport['ARPLatitude'].apply(convert_tude)

#CONVERT ACTIVATION DATES AND FILTER DATAFRAME ACCORDINGLY
airport['ActivationDate'] = pd.to_datetime(airport['ActiviationDate'].astype('str'), errors='coerce')
airport = airport[airport['ActivationDate'] >= pd.Timestamp('1950-01-01')]

#SLICE DATAFRAME FOR RELEVANT INFORMATION
airport = airport[['StateName','Lon','Lat','ActivationDate']]

In [22]:
military.head()

,name,longitude,latitude
0,Sierra Army Depot,-120.152620,40.266163
1,"Fort Derussy, Hale Koa Hotel",-157.835044,21.282099
2,"USCG Air Station Barbers Point (E,H)",-158.069899,21.300353
3,"USCG Base Support Unit Honolulu (E,F)",-157.874273,21.305771
4,"Hickam AFB (E,C,F,H)",-157.944116,21.340868
